<a href="https://colab.research.google.com/github/omniaghazy/Core_X/blob/main/Friction_loss_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.7-cp312-cp312-linux_x86_64.whl size=99873466 sha256=2ff8f36180e360bf307aa114caef8d164e0ff5cdd4401a980eb2dea33085870d
  Stored in directory: /root/.cache/pip/wheels/72/95/1d/b336e5ee612ae9a019bfff4dc0bedd100ee6f0570db205fdf8
Successfully built pybullet


In [ ]:
import pybullet as p
import time
import pybullet_data
import math
import pandas as pd
import os
import numpy as np # ضروري لو استخدمنا math.sin
from typing import List, Dict # عشان شكل الـCode يكون منظم

print("Libraries installed and imported successfully.")

# ==========================================================
# ⚙️ (ملغي) الـBacklash Injector مش محتاجينه ⚙️
# ==========================================================

# ==========================================================
# الخطوة 2: إعداد بيئة المحاكاة (زي ما هو)
# ==========================================================
# (هنا هييجي الـSetup بتاع الـconnect والـloadURDF)
print("\nStarting PyBullet simulation in HEADLESS (DIRECT) mode...")
# ... (نفس الـCode بتاع الـSetup لغاية ما الـrobotId يتحمل)

physicsClient = p.connect(p.DIRECT)
pybullet_data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data_path)
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.01]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
# *ملاحظة: لو الـKUKA ما تحملش، هنغيره للـR2D2 زي ما اتفقنا قبل كده*
robotId = p.loadURDF("kuka_iiwa/model.urdf", startPos, startOrientation, useFixedBase=True)
print(f"Successfully loaded KUKA LBR iiwa (Robot ID: {robotId})")

# --- 4. إعداد المفاصل ---
numJoints = p.getNumJoints(robotId)
controllable_joints = [i for i in range(numJoints) if p.getJointInfo(robotId, i)[2] == p.JOINT_REVOLUTE]
end_effector_link_index = 6
for joint in controllable_joints:
    # (جديد) تفعيل الـForce/Torque Sensor عشان نسجل الـTorque اللي هيزيد
    p.enableJointForceTorqueSensor(robotId, joint, enableSensor=1)
    p.setJointMotorControl2(robotId, joint, p.VELOCITY_CONTROL, force=0)


# --- 5. إعداد "حقن العطل" (Friction Loss) ---
joint_to_fail = 4 # المفصل رقم 4 هو اللي هيتآكل
INITIAL_FRICTION = 0.5   # قيمة الاحتكاك الأولية
MAX_FRICTION = 2.0       # أقصى قيمة هيوصلها الاحتكاك (لتدهور واضح)
FRICTION_INCREMENT = (MAX_FRICTION - INITIAL_FRICTION) / 10000.0 # الزيادة التدريجية

data_log: List[List] = []
total_steps = 20000
fault_injection_step = 10000

print(f"Simulation starting for {total_steps} steps (Friction Loss)...")
print(f" - FAULT (Friction Increase) will be injected in JOINT {joint_to_fail} starting at step {fault_injection_step}.")

# --- 6. اللوب الرئيسي للمحاكاة (توليد البيانات) ---
start_time = time.time()
try:
    current_friction = INITIAL_FRICTION # نبدأ بالاحتكاك الطبيعي

    for step in range(total_steps):

        # أ. حساب "المسار" المستهدف (Trajectory)
        target_x = 0.6 + 0.2 * math.sin(step * 0.01)
        target_y = 0.0
        target_z = 0.8
        target_position = [target_x, target_y, target_z]

        # ب. حساب الأوامر المثالية (Inverse Kinematics)
        perfect_commands_list = p.calculateInverseKinematics(
            robotId, end_effector_link_index, target_position
        )

        # --- ج. (حقن العطل) ---
        if step >= fault_injection_step:
            if step == fault_injection_step:
                print(f"\n[STEP {step}] --- FAULT INJECTED! Starting Friction Increase for JOINT {joint_to_fail} ---\n")

            # 1. زيادة قيمة الاحتكاك تدريجياً
            current_friction += FRICTION_INCREMENT

            # 2. تطبيق الـChange Dynamics (باستخدام lateralFriction)
            p.changeDynamics(
                bodyUniqueId=robotId,
                linkIndex=joint_to_fail,
                lateralFriction=min(current_friction, MAX_FRICTION) # نثبت القيمة عند الـMAX
            )

        log_row = [step] # بداية صف البيانات

        # --- لوب على كل المفاصل ---
        for i in range(len(controllable_joints)):
            joint_index = controllable_joints[i]

            # د. استخراج الأمر المثالي (بنبعت الأمر المثالي مباشرة)
            target_pos = perfect_commands_list[i]

            # هـ. إرسال الأمر للمحاكي (هنا مفيش عطل في الـCommand)
            p.setJointMotorControl2(
                bodyIndex=robotId,
                jointIndex=joint_index,
                controlMode=p.POSITION_CONTROL,
                targetPosition=target_pos, # الأمر مثالي، الـPhysics هي اللي هتتغير
                positionGain=0.1,
                velocityGain=0.5
            )

            # و. تسجيل البيانات
            actual_position = p.getJointState(robotId, joint_index)[0]
            # (جديد) تسجيل الـTorque المُتولّد (الـPM Signal)
            actual_torque = p.getJointState(robotId, joint_index)[3] # [3] هو الـApplied Torque

            log_row.append(target_pos)
            log_row.append(actual_position)
            log_row.append(actual_torque) # أضفنا الـTorque هنا

        log_row.append(current_friction) # بنسجل قيمة الـFriction الحالية كـLabel
        data_log.append(log_row)
        p.stepSimulation() # تشغيل خطوة المحاكاة

# --- 7. إنهاء الاتصال وحفظ البيانات ---
finally:
    end_time = time.time()
    p.disconnect()
    # ... (نفس الـCode بتاع حفظ الـDataFrame)

    if data_log:
        print(f"Logged {len(data_log)} data points.")
        columns = ['Step']
        for j in controllable_joints:
            columns.append(f'Cmd_J{j}')
            columns.append(f'Act_J{j}')
            columns.append(f'Trq_J{j}') # عمود جديد للـTorque
        columns.append('Friction_Label')

        df = pd.DataFrame(data_log, columns=columns)
        output_filename = "trajectory_friction_data.csv"
        df.to_csv(output_filename, index=False)

        print("\n========================================================")
        print(f"✅ SUCCESS! Data saved to: {os.path.abspath(output_filename)}")
        print(f"Columns: {columns}")
    else:
        print("No data was logged.")

Libraries installed and imported successfully.

Starting PyBullet simulation in HEADLESS (DIRECT) mode...
Successfully loaded KUKA LBR iiwa (Robot ID: 1)
Simulation starting for 20000 steps (Friction Loss)...
 - FAULT (Friction Increase) will be injected in JOINT 4 starting at step 10000.

[STEP 10000] --- FAULT INJECTED! Starting Friction Increase for JOINT 4 ---

Logged 20000 data points.

✅ SUCCESS! Data saved to: /content/trajectory_friction_data.csv
Columns: ['Step', 'Cmd_J0', 'Act_J0', 'Trq_J0', 'Cmd_J1', 'Act_J1', 'Trq_J1', 'Cmd_J2', 'Act_J2', 'Trq_J2', 'Cmd_J3', 'Act_J3', 'Trq_J3', 'Cmd_J4', 'Act_J4', 'Trq_J4', 'Cmd_J5', 'Act_J5', 'Trq_J5', 'Cmd_J6', 'Act_J6', 'Trq_J6', 'Friction_Label']


In [ ]:
df = pd.read_csv("/content/trajectory_friction_data.csv")
df.head()

,Step,Cmd_J0,Act_J0,Trq_J0,Cmd_J1,Act_J1,Trq_J1,Cmd_J2,Act_J2,Trq_J2,...,Cmd_J4,Act_J4,Trq_J4,Cmd_J5,Act_J5,Trq_J5,Cmd_J6,Act_J6,Trq_J6,Friction_Label
0,0,-3.914271e-12,0.000000,0.000000,0.501474,0.000000,0.000000,-1.241552e-12,0.000000,0.000000,...,8.906893e-13,0.000000e+00,0.000000,0.169378,0.000000,0.000000,0.000000,0.000000,0.000000,0.5
1,1,1.274712e-05,-0.000014,153.895114,0.470348,0.050061,15231.858390,-2.275887e-05,-0.000033,182.525885,...,3.487295e-07,3.832701e-08,111.219555,0.183607,0.016939,227.385174,-0.000068,-0.000068,-0.003978,0.5
2,2,5.604404e-06,0.000181,60.286766,0.436917,0.117884,5873.631744,1.011939e-05,0.000084,70.679386,...,-4.887801e-04,-4.927337e-04,43.429638,0.198424,0.041537,94.666786,0.000138,0.000138,-0.003400,0.5
3,3,-1.222840e-04,0.000571,3.560761,0.412631,0.186939,510.897594,2.890419e-04,0.000620,5.076661,...,-2.250847e-03,-2.270896e-03,5.458987,0.213416,0.071284,20.885205,0.000662,0.000662,0.010873,0.5
4,4,-3.666178e-04,0.000812,-27.949765,0.396956,0.249176,-2215.060530,7.839042e-04,0.001390,-29.840921,...,-4.883045e-03,-4.930950e-03,-13.128391,0.229414,0.103612,-16.138823,0.001261,0.001261,0.034670,0.5


In [ ]:
df.describe()

,Step,Cmd_J0,Act_J0,Trq_J0,Cmd_J1,Act_J1,Trq_J1,Cmd_J2,Act_J2,Trq_J2,...,Cmd_J4,Act_J4,Trq_J4,Cmd_J5,Act_J5,Trq_J5,Cmd_J6,Act_J6,Trq_J6,Friction_Label
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,9999.500000,-0.033744,-0.033757,-0.000708,1.322522,1.324030,-41.562682,-0.078092,-0.078151,0.352080,...,0.107604,0.107608,-0.172643,-0.186568,-0.186917,-0.100608,0.003943,0.003943,-0.000222,0.875038
std,5773.647028,0.043406,0.043415,2.670629,0.469551,0.470415,127.016585,0.026712,0.026727,2.033458,...,0.075544,0.075558,1.951381,0.476490,0.476719,1.861055,0.104046,0.104046,6.911635,0.484164
min,0.000000,-0.088395,-0.088446,-43.638497,-0.193522,-0.193469,-3455.263586,-0.141014,-0.141236,-46.915617,...,-0.021837,-0.021857,-21.341000,-1.909242,-1.911023,-35.120468,-0.100925,-0.100925,-7.224786,0.500000
25%,4999.750000,-0.065573,-0.065589,-1.499869,1.181496,1.184028,-47.930868,-0.095992,-0.096087,-0.674555,...,0.041164,0.041131,-1.963474,-0.411433,-0.412195,-0.290377,-0.100083,-0.100083,-6.876238,0.500000
50%,9999.500000,-0.046387,-0.046383,0.022193,1.511471,1.513328,-43.641036,-0.072637,-0.072692,0.436446,...,0.103475,0.103503,-0.156489,-0.021444,-0.021950,-0.108972,0.002278,0.002278,0.001246,0.500075
75%,14999.250000,-0.010230,-0.010361,1.394410,1.659200,1.661108,-38.332998,-0.063301,-0.063358,1.437019,...,0.174576,0.174478,1.623051,0.183188,0.183122,0.067195,0.108250,0.108250,6.876123,1.250038
max,19999.000000,0.142252,0.142074,153.895114,1.690607,1.692758,15231.858390,0.002237,0.003014,182.525885,...,0.294564,0.294976,111.219555,0.303121,0.304012,227.385174,0.108576,0.108576,7.230582,2.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Step            20000 non-null  int64  
 1   Cmd_J0          20000 non-null  float64
 2   Act_J0          20000 non-null  float64
 3   Trq_J0          20000 non-null  float64
 4   Cmd_J1          20000 non-null  float64
 5   Act_J1          20000 non-null  float64
 6   Trq_J1          20000 non-null  float64
 7   Cmd_J2          20000 non-null  float64
 8   Act_J2          20000 non-null  float64
 9   Trq_J2          20000 non-null  float64
 10  Cmd_J3          20000 non-null  float64
 11  Act_J3          20000 non-null  float64
 12  Trq_J3          20000 non-null  float64
 13  Cmd_J4          20000 non-null  float64
 14  Act_J4          20000 non-null  float64
 15  Trq_J4          20000 non-null  float64
 16  Cmd_J5          20000 non-null  float64
 17  Act_J5          20000 non-null 